In [1]:
/FileStore/tables/train.csv

In [2]:
#INPUT: train.csv
#Output: 
"""

1. Clean up wrong X and Y values (very few of them)

2. visualize data.

2. Parse input to get features: for e.g: get date, time, year, month, etc..)

3. Select, and generate features.

3. Remove outliers.

4. do PCA

Output: train dataframe with features and labels column

        test dataframe with features and lables column

        visuals to provide insights on data that help select, and tune the models.       

 a toolbox list to choose from:

         Typical graphical techniques used in EDA are


Box plot

Histogram

Multi-vari chart

Run chart

Pareto chart

Scatter plot

Stem-and-leaf plot

Parallel coordinates

Odds ratio

Targeted projection pursuit

Glyph-based visualization methods such as PhenoPlot[8] and Chernoff faces

Projection methods such as grand tour, guided tour and manual tour

Interactive versions of these plots

        Dimensionality reduction:

Multidimensional scaling

Principal component analysis (PCA)

Multilinear PCA

Nonlinear dimensionality reduction (NLDR)

        Typical quantitative techniques are:
Median polish

Trimean

Ordination

History

        
"""

In [3]:
filename="/FileStore/tables/train.csv"
data=spark.read.csv(filename, header=True, inferSchema=True)
print(data.count())
print(len(data.columns))
data.printSchema() #the data was inferred properly. Class is an int. Features are double.

878049
9
root
-- Dates: timestamp (nullable = true)
-- Category: string (nullable = true)
-- Descript: string (nullable = true)
-- DayOfWeek: string (nullable = true)
-- PdDistrict: string (nullable = true)
-- Resolution: string (nullable = true)
-- Address: string (nullable = true)
-- X: double (nullable = true)
-- Y: double (nullable = true)

In [4]:
"""
Dates - timestamp of the crime incident
Category - category of the crime incident (only in train.csv). This is the target variable you are going to predict.
Descript - detailed description of the crime incident (only in train.csv)
DayOfWeek - the day of the week
PdDistrict - name of the Police Department District
Resolution - how the crime incident was resolved (only in train.csv)
Address - the approximate street address of the crime incident 
X - Longitude
Y - Latitude
"""

Out[2]: '\nDates - timestamp of the crime incident\nCategory - category of the crime incident (only in train.csv). This is the target variable you are going to predict.\nDescript - detailed description of the crime incident (only in train.csv)\nDayOfWeek - the day of the week\nPdDistrict - name of the Police Department District\nResolution - how the crime incident was resolved (only in train.csv)\nAddress - the approximate street address of the crime incident \nX - Longitude\nY - Latitude\n'

In [5]:
#Parsing the time column to generate features, year, month, day, hour, season
from pyspark.sql.functions import col, hour, minute, second, year, month, dayofmonth, date_format

dateFormat = "%Y-%m-%d %H:%M:%S"
time_features=data.select(col("Dates"))
time_features = time_features.withColumn("hour", hour(col("Dates"))).withColumn("minute", minute(col("Dates"))).withColumn("second", second(col("Dates"))).withColumn("dayOfMonth", dayofmonth(col("Dates"))).withColumn("year", year(col("Dates"))).withColumn("month", month(col("Dates"))).withColumn("dayOfWeek", date_format(col("Dates"), "EEEE")).withColumn("weekOfMonth", date_format(col("Dates"), "W"))
time_features.show(5)
time_features.printSchema()

+-------------------+----+------+------+----------+----+-----+---------+-----------+
 Dates|hour|minute|second|dayOfMonth|year|month|dayOfWeek|weekOfMonth|
+-------------------+----+------+------+----------+----+-----+---------+-----------+
2015-05-13 23:53:00| 23| 53| 0| 13|2015| 5|Wednesday| 3|
2015-05-13 23:53:00| 23| 53| 0| 13|2015| 5|Wednesday| 3|
2015-05-13 23:33:00| 23| 33| 0| 13|2015| 5|Wednesday| 3|
2015-05-13 23:30:00| 23| 30| 0| 13|2015| 5|Wednesday| 3|
2015-05-13 23:30:00| 23| 30| 0| 13|2015| 5|Wednesday| 3|
+-------------------+----+------+------+----------+----+-----+---------+-----------+
only showing top 5 rows

root
-- Dates: timestamp (nullable = true)
-- hour: integer (nullable = true)
-- minute: integer (nullable = true)
-- second: integer (nullable = true)
-- dayOfMonth: integer (nullable = true)
-- year: integer (nullable = true)
-- month: integer (nullable = true)
-- dayOfWeek: string (nullable = true)
-- weekOfMonth: string (nullable = true)